In [1]:
from collections import deque
import model
from dataUtils import *
from logger import MyLogger
import sys
import PTB_data_reader
import time
import numpy as np
import lstm_char_cnn
import config
import pickle
import dataloader
import importlib
from tensorflow import graph_util

tf.logging.set_verbosity(tf.logging.ERROR)

logger = MyLogger("ERDMain")
tf.app.flags.DEFINE_string('f', '', 'kernel')

W0830 12:51:26.128711 140337803204416 deprecation_wrapper.py:119] From /home/hadoop/ERD/model.py:6: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction instead.

Using TensorFlow backend.


In [2]:

#load sentiment analysis data
sentiReader = dataloader.SentiDataLoader(
                                        dirpath = '/home/hadoop/trainingandtestdata',
                                        trainfile = 'training.1600000.processed.noemoticon.csv', 
                                        testfile = 'testdata.manual.2009.06.14.csv', 
                                        charVocab = char_vocab
                        )
# sentiReader.load_data()
sentiReader.load_data_fast(
                        '/home/hadoop/ERD/data/senti_train_data.pickle',
                        '/home/hadoop/ERD/data/senti_train_label.pickle',
                        '/home/hadoop/ERD/data/senti_test_data.pickle',
                        '/home/hadoop/ERD/data/senti_test_label.pickle'
                          )


In [3]:
#load PTB data
# word_vocab, char_vocab, word_tensors, char_tensors, max_word_length = \
#     PTB_data_reader.load_data(FLAGS.data_dir, FLAGS.max_word_length, char_vocab, eos=FLAGS.EOS)
word_vocab, char_vocab, word_tensors, char_tensors = \
    PTB_data_reader.load_data_fast()
max_word_length = FLAGS.max_word_length
train_reader = PTB_data_reader.DataReader(word_tensors['train'], char_tensors['train'],
                          FLAGS.batch_size, FLAGS.num_unroll_steps) 


In [ ]:

# load twitter data
# load_data(FLAGS.data_file_path)
load_data_fast()

# (self, input_dim, hidden_dim, max_seq_len, max_word_num, class_num, action_num):
print(  FLAGS.embedding_dim, FLAGS.hidden_dim, 
            FLAGS.max_seq_len, FLAGS.max_sent_len, 
                FLAGS.class_num, FLAGS.action_num   )
logger.info(    (FLAGS.embedding_dim, FLAGS.hidden_dim, 
                    FLAGS.max_seq_len, FLAGS.max_sent_len, 
                        FLAGS.class_num, FLAGS.action_num)  )

print(get_curtime() + " Data loaded.")
logger.info(get_curtime() + " Data loaded.")

In [4]:
importlib.reload(lstm_char_cnn)

<module 'lstm_char_cnn' from '/home/hadoop/ERD/lstm_char_cnn.py'>

In [5]:
with tf.Graph().as_default() as char_graph:
    with tf.Session(graph=char_graph) as sess:
        w2v = lstm_char_cnn.WordEmbedding(
                    max_word_length = FLAGS.max_char_num , 
                    char_vocab_size = char_vocab.size, 
                    char_embed_size = FLAGS.char_embed_size, 
                    kernels = eval(FLAGS.kernels), 
                    kernel_features = eval(FLAGS.kernel_features), 
                    num_highway_layers = FLAGS.highway_layers,
                    embedding_dim = FLAGS.embedding_dim
                )
        lstm_lm = lstm_char_cnn.LSTM_LM(
                    batch_size = FLAGS.batch_size, 
                    num_unroll_steps = FLAGS.num_unroll_steps,
                    rnn_size = 300, 
                    num_rnn_layers = FLAGS.rnn_layers, 
                    word_vocab_size = word_vocab.size
                )
        char_train_graph = lstm_char_cnn.infer_train_model(
                            w2v, lstm_lm, 
                            batch_size = FLAGS.batch_size, 
                            num_unroll_steps = FLAGS.num_unroll_steps, 
                            max_word_length = FLAGS.max_char_num, 
                            learning_rate = FLAGS.learning_rate,
                            max_grad_norm = FLAGS.max_grad_norm
                         )
        t_vals = tf.trainable_variables()
        g_vals = tf.global_variables()
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=4)
        sess.run(tf.global_variables_initializer())
        summary_writer = tf.summary.FileWriter(FLAGS.train_dir, graph=sess.graph)
        lstm_char_cnn.Train_Char_Model(sess, char_train_graph, train_reader, saver, summary_writer)
        
        c_g_def = graph_util.convert_variables_to_constants(
        sess,
        char_graph.as_graph_def(),
        ["lstm_word_embedding", "lstm_sent_embedding"],
        variable_names_whitelist=None,
        variable_names_blacklist=t_vals)
        c_g_def = char_graph.as_graph_def()

input_: Tensor("input:0", shape=(20, 35, 21), dtype=int32)
input_cnn: Tensor("Embedding_1/CNN_OUT/add_7:0", shape=(700, 1100), dtype=float32)
     5: 0 [    5/ 1327], train_loss/perplexity = 9.15713978/9481.8974609 secs/batch = 0.6161s, grad.norm=0.11757042
    10: 0 [   10/ 1327], train_loss/perplexity = 9.07616043/8744.3271484 secs/batch = 0.6236s, grad.norm=0.14040542
    15: 0 [   15/ 1327], train_loss/perplexity = 8.97761822/7923.7368164 secs/batch = 0.7529s, grad.norm=0.17943056
    20: 0 [   20/ 1327], train_loss/perplexity = 8.69668484/5983.0444336 secs/batch = 0.6355s, grad.norm=0.39473239
    25: 0 [   25/ 1327], train_loss/perplexity = 7.77400303/2377.9714355 secs/batch = 0.6412s, grad.norm=0.67980880
    30: 0 [   30/ 1327], train_loss/perplexity = 7.22767782/1377.0211182 secs/batch = 0.6140s, grad.norm=0.46658877
    35: 0 [   35/ 1327], train_loss/perplexity = 7.45424461/1727.1788330 secs/batch = 0.6122s, grad.norm=0.63674241
    40: 0 [   40/ 1327], train_loss/perplexity

   355: 0 [  355/ 1327], train_loss/perplexity = 6.71332455/823.3032227 secs/batch = 0.6169s, grad.norm=0.69381249
   360: 0 [  360/ 1327], train_loss/perplexity = 6.76541567/867.3266602 secs/batch = 0.6074s, grad.norm=0.37876713
   365: 0 [  365/ 1327], train_loss/perplexity = 6.68111181/797.2049561 secs/batch = 0.6213s, grad.norm=0.41956562
   370: 0 [  370/ 1327], train_loss/perplexity = 6.79958057/897.4707642 secs/batch = 0.6053s, grad.norm=0.53257513
   375: 0 [  375/ 1327], train_loss/perplexity = 6.51513338/675.2839966 secs/batch = 0.6098s, grad.norm=0.39017209
   380: 0 [  380/ 1327], train_loss/perplexity = 6.68172073/797.6905518 secs/batch = 0.6667s, grad.norm=0.45927042
   385: 0 [  385/ 1327], train_loss/perplexity = 6.84981155/943.7030640 secs/batch = 0.7422s, grad.norm=0.49560401
   390: 0 [  390/ 1327], train_loss/perplexity = 6.61472511/745.9996338 secs/batch = 0.7155s, grad.norm=0.36983743
   395: 0 [  395/ 1327], train_loss/perplexity = 6.82563639/921.1624146 secs/bat

   715: 0 [  715/ 1327], train_loss/perplexity = 6.61808443/748.5098877 secs/batch = 0.6127s, grad.norm=0.37769234
   720: 0 [  720/ 1327], train_loss/perplexity = 6.63069248/758.0068970 secs/batch = 0.6114s, grad.norm=0.47281492
   725: 0 [  725/ 1327], train_loss/perplexity = 6.42002869/614.0207520 secs/batch = 0.6069s, grad.norm=0.58388084
   730: 0 [  730/ 1327], train_loss/perplexity = 6.45088720/633.2638550 secs/batch = 0.6132s, grad.norm=0.32802877
   735: 0 [  735/ 1327], train_loss/perplexity = 6.58612442/724.9657593 secs/batch = 0.6158s, grad.norm=0.30253953
   740: 0 [  740/ 1327], train_loss/perplexity = 6.46294165/640.9437256 secs/batch = 0.6105s, grad.norm=0.44014961
   745: 0 [  745/ 1327], train_loss/perplexity = 6.61271048/744.4982300 secs/batch = 0.6219s, grad.norm=0.40359932
   750: 0 [  750/ 1327], train_loss/perplexity = 6.45377731/635.0967407 secs/batch = 0.6074s, grad.norm=0.40300721
   755: 0 [  755/ 1327], train_loss/perplexity = 6.49482632/661.7092896 secs/bat

  1075: 0 [ 1075/ 1327], train_loss/perplexity = 6.55407906/702.1022339 secs/batch = 0.6192s, grad.norm=0.31029329
  1080: 0 [ 1080/ 1327], train_loss/perplexity = 6.46852350/644.5313721 secs/batch = 0.6274s, grad.norm=0.31953108
  1085: 0 [ 1085/ 1327], train_loss/perplexity = 6.51386738/674.4296265 secs/batch = 0.6151s, grad.norm=0.41981539
  1090: 0 [ 1090/ 1327], train_loss/perplexity = 6.59781122/733.4879761 secs/batch = 0.6121s, grad.norm=0.25253704
  1095: 0 [ 1095/ 1327], train_loss/perplexity = 6.57855415/719.4982910 secs/batch = 0.6093s, grad.norm=0.43054444
  1100: 0 [ 1100/ 1327], train_loss/perplexity = 6.64365864/767.8993530 secs/batch = 0.6096s, grad.norm=0.43237484
  1105: 0 [ 1105/ 1327], train_loss/perplexity = 6.45963526/638.8280029 secs/batch = 0.6088s, grad.norm=0.38816988
  1110: 0 [ 1110/ 1327], train_loss/perplexity = 6.77248955/873.4837646 secs/batch = 0.6179s, grad.norm=0.32356152
  1115: 0 [ 1115/ 1327], train_loss/perplexity = 6.52636671/682.9124756 secs/bat

  1432: 1 [  105/ 1327], train_loss/perplexity = 6.63615036/762.1553345 secs/batch = 0.6131s, grad.norm=0.32633582
  1437: 1 [  110/ 1327], train_loss/perplexity = 6.44774961/631.2800903 secs/batch = 0.6181s, grad.norm=0.34873429
  1442: 1 [  115/ 1327], train_loss/perplexity = 6.27517796/531.2208862 secs/batch = 0.6096s, grad.norm=0.40628150
  1447: 1 [  120/ 1327], train_loss/perplexity = 6.55820513/705.0051880 secs/batch = 0.6134s, grad.norm=0.38063699
  1452: 1 [  125/ 1327], train_loss/perplexity = 6.64219093/766.7731323 secs/batch = 0.6144s, grad.norm=0.37956536
  1457: 1 [  130/ 1327], train_loss/perplexity = 6.61789131/748.3653564 secs/batch = 0.6186s, grad.norm=0.48556170
  1462: 1 [  135/ 1327], train_loss/perplexity = 6.49723959/663.3081055 secs/batch = 0.6116s, grad.norm=0.40482077
  1467: 1 [  140/ 1327], train_loss/perplexity = 6.59050989/728.1520386 secs/batch = 0.6161s, grad.norm=0.29561308
  1472: 1 [  145/ 1327], train_loss/perplexity = 6.67261648/790.4611206 secs/bat

...

 21827: 16 [  595/ 1327], train_loss/perplexity = 4.50998783/90.9207153 secs/batch = 0.6175s, grad.norm=0.68956125
 21832: 16 [  600/ 1327], train_loss/perplexity = 4.80248260/121.8124542 secs/batch = 0.6173s, grad.norm=0.69121289
 21837: 16 [  605/ 1327], train_loss/perplexity = 4.74881887/115.4478455 secs/batch = 0.6545s, grad.norm=0.65481025
 21842: 16 [  610/ 1327], train_loss/perplexity = 4.81174612/122.9461060 secs/batch = 0.6117s, grad.norm=0.68667269
 21847: 16 [  615/ 1327], train_loss/perplexity = 4.38686466/80.3879776 secs/batch = 0.6175s, grad.norm=0.65373635
 21852: 16 [  620/ 1327], train_loss/perplexity = 4.67983818/107.7526321 secs/batch = 0.6055s, grad.norm=0.66042912
 21857: 16 [  625/ 1327], train_loss/perplexity = 4.77119350/118.0600662 secs/batch = 0.6069s, grad.norm=0.65939325
 21862: 16 [  630/ 1327], train_loss/perplexity = 4.85489845/128.3676605 secs/batch = 0.6112s, grad.norm=0.65901059
 21867: 16 [  635/ 1327], train_loss/perplexity = 4.56235504/95.8088455 se

 22187: 16 [  955/ 1327], train_loss/perplexity = 4.62050724/101.5455246 secs/batch = 0.6054s, grad.norm=0.65879548
 22192: 16 [  960/ 1327], train_loss/perplexity = 4.92002439/137.0059509 secs/batch = 0.6114s, grad.norm=0.68970180
 22197: 16 [  965/ 1327], train_loss/perplexity = 4.60507298/99.9902802 secs/batch = 0.6149s, grad.norm=0.63094169
 22202: 16 [  970/ 1327], train_loss/perplexity = 4.87729692/131.2753296 secs/batch = 0.6053s, grad.norm=0.65094095
 22207: 16 [  975/ 1327], train_loss/perplexity = 4.56266975/95.8390045 secs/batch = 0.6099s, grad.norm=0.66320819
 22212: 16 [  980/ 1327], train_loss/perplexity = 4.37678862/79.5820541 secs/batch = 0.6103s, grad.norm=0.65513480
 22217: 16 [  985/ 1327], train_loss/perplexity = 4.56652641/96.2093353 secs/batch = 0.6104s, grad.norm=0.67987096
 22222: 16 [  990/ 1327], train_loss/perplexity = 4.70859289/110.8960037 secs/batch = 0.6088s, grad.norm=0.68398112
 22227: 16 [  995/ 1327], train_loss/perplexity = 4.72895002/113.1766663 sec

 22547: 16 [ 1315/ 1327], train_loss/perplexity = 4.81970882/123.9290009 secs/batch = 0.6129s, grad.norm=0.66378713
 22552: 16 [ 1320/ 1327], train_loss/perplexity = 4.79762077/121.2216568 secs/batch = 0.6112s, grad.norm=0.65765280
 22557: 16 [ 1325/ 1327], train_loss/perplexity = 4.66536188/106.2040100 secs/batch = 0.6092s, grad.norm=0.66260576
Epoch training time: 812.98597407341
Saved char model cv/epoch016_4.6040.model
 22564: 17 [    5/ 1327], train_loss/perplexity = 4.75396299/116.0432510 secs/batch = 0.6030s, grad.norm=0.68925190
 22569: 17 [   10/ 1327], train_loss/perplexity = 4.32033634/75.2139206 secs/batch = 0.6101s, grad.norm=0.65024364
 22574: 17 [   15/ 1327], train_loss/perplexity = 4.59172535/98.6645126 secs/batch = 0.6273s, grad.norm=0.65584475
 22579: 17 [   20/ 1327], train_loss/perplexity = 4.76041412/116.7942810 secs/batch = 0.6089s, grad.norm=0.67775625
 22584: 17 [   25/ 1327], train_loss/perplexity = 4.66079330/105.7199173 secs/batch = 0.6153s, grad.norm=0.6764

 22904: 17 [  345/ 1327], train_loss/perplexity = 4.54998398/94.6308899 secs/batch = 0.6838s, grad.norm=0.65178490
 22909: 17 [  350/ 1327], train_loss/perplexity = 4.58312321/97.8194275 secs/batch = 0.6711s, grad.norm=0.67524099
 22914: 17 [  355/ 1327], train_loss/perplexity = 4.58417749/97.9226151 secs/batch = 0.7295s, grad.norm=0.65750480
 22919: 17 [  360/ 1327], train_loss/perplexity = 4.69108582/108.9714355 secs/batch = 0.6103s, grad.norm=0.68432397
 22924: 17 [  365/ 1327], train_loss/perplexity = 4.63217020/102.7367783 secs/batch = 0.6874s, grad.norm=0.65962607
 22929: 17 [  370/ 1327], train_loss/perplexity = 4.71683979/111.8143387 secs/batch = 0.6114s, grad.norm=0.71993673
 22934: 17 [  375/ 1327], train_loss/perplexity = 4.12627745/61.9468918 secs/batch = 0.6451s, grad.norm=0.65077734
 22939: 17 [  380/ 1327], train_loss/perplexity = 4.22449255/68.3398132 secs/batch = 0.6654s, grad.norm=0.68256313
 22944: 17 [  385/ 1327], train_loss/perplexity = 4.44114017/84.8716583 secs/

KeyboardInterrupt: 

In [21]:
t_vals

[<tf.Variable 'Embedding/char_embedding:0' shape=(72, 15) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_1/w:0' shape=(1, 1, 15, 50) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_1/b:0' shape=(50,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_2/w:0' shape=(1, 2, 15, 100) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_2/b:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_3/w:0' shape=(1, 3, 15, 150) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_3/b:0' shape=(150,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_4/w:0' shape=(1, 4, 15, 200) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_4/b:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_5/w:0' shape=(1, 5, 15, 200) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_5/b:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_6/w:0' shape=(1, 6, 15, 200) dtype=float32_ref>,
 <tf.Variable 'Embeddin

In [20]:
g_vals

[<tf.Variable 'Embedding/char_embedding:0' shape=(72, 15) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_1/w:0' shape=(1, 1, 15, 50) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_1/b:0' shape=(50,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_2/w:0' shape=(1, 2, 15, 100) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_2/b:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_3/w:0' shape=(1, 3, 15, 150) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_3/b:0' shape=(150,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_4/w:0' shape=(1, 4, 15, 200) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_4/b:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_5/w:0' shape=(1, 5, 15, 200) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_5/b:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_6/w:0' shape=(1, 6, 15, 200) dtype=float32_ref>,
 <tf.Variable 'Embeddin

In [6]:
from tensorflow.losses import Reduction

In [7]:
with tf.Graph().as_default() as senti_graph:
    with tf.Session(graph=senti_graph) as sess:
        s_model = model.SentiModel(FLAGS.hidden_dim, 5)
#         sent_x = tf.placeholder(tf.int32, shape = [None, 31, 21])
        sent_y = tf.placeholder(tf.float32, shape = [None, 3])
#         words2vec = char_model(sent_x) #[None, max_word_num, kernerl_size]
#         sentence = tf.reshape(words2vec, [-1, max_word_num, embedding_dim])
        sentence = tf.placeholder(tf.float32, shape = [None, 31, 300], name = "sentence_input")
#         sentence, = tf.import_graph_def(c_g_def, input_map={"input:0":sent_x}, return_elements=["lstm_word_embedding"])
        senti_features = s_model(sentence)
        senti_out = tf.identity(senti_features, "senti_feature_map")
        classifier = tf.layers.Dense(3, activation=tf.nn.relu, trainable=True)
        senti_rst = classifier(senti_features)
        sent_scores = tf.nn.softmax(senti_rst, axis=1)
        sent_pred = tf.argmax(sent_scores, 1, name="predictions")
        sent_loss = tf.losses.softmax_cross_entropy(
                            sent_y,
                            senti_rst,
                            weights=1.0,
                            label_smoothing=0,
                            scope=None,
                            loss_collection=tf.GraphKeys.LOSSES,
                            reduction=Reduction.SUM_BY_NONZERO_WEIGHTS
                        )
        sent_correct_predictions = tf.equal(sent_pred, tf.argmax(sent_y, 1))
        sent_acc = tf.reduce_mean(tf.cast(sent_correct_predictions, "float"), name="accuracy")
        sent_global_step = tf.Variable(0, name="global_step", trainable=False)
        sent_train_op = tf.train.AdagradOptimizer(0.01).minimize(sent_loss, sent_global_step)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=4)
        sess.run(tf.global_variables_initializer())
        summary_writer = tf.summary.FileWriter(FLAGS.train_dir, graph=sess.graph)
#         lstm_char_cnn.Train_Char_Model(sess, char_train_graph, train_reader, saver, summary_writer)
        t_vals = tf.trainable_variables()
#         s_g_def = graph_util.convert_variables_to_constants(
#         sess,
#         senti_graph.as_graph_def(),
#         ["senti_feature_map"],
#         variable_names_whitelist=None,
#         variable_names_blacklist=t_vals)
        s_g_def = senti_graph.as_graph_def()
        

In [15]:
with tf.Graph().as_default() as g_combined:
    with tf.Session(graph=g_combined) as sess:
        sent_x = tf.placeholder(tf.int32, shape = [20, 31, 21], name = "sent_input")
        dp = tf.placeholder(tf.float32)
        sentence, = tf.import_graph_def(c_g_def, input_map={"input:0":sent_x, "LSTM/Dropout:0":dp}, return_elements=["lstm_word_embedding:0"])
        senti_F, = tf.import_graph_def(s_g_def, input_map={"sentence_input:0":sentence}, return_elements=["senti_feature_map:0"])
        data_X, data_Y = sentiReader.GetTrainingBatch(
                                            0, 
                                            20
                            )
        sess.run(tf.global_variables_initializer())
        tt_vals = tf.trainable_variables()
        print("tt_vals:", tt_vals)
        scores = sess.run(senti_F,
               feed_dict={sent_x: data_X, dp:0.8})

tt_vals: []


FailedPreconditionError: Attempting to use uninitialized value import/Embedding/TDNN/kernel_6/b
	 [[node import/Embedding/TDNN/kernel_6/b/read (defined at <ipython-input-15-1e464ab14f9e>:5) ]]

Original stack trace for 'import/Embedding/TDNN/kernel_6/b/read':
  File "/home/hadoop/.conda/envs/TF/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hadoop/.conda/envs/TF/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hadoop/.conda/envs/TF/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hadoop/.conda/envs/TF/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/hadoop/.conda/envs/TF/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/hadoop/.conda/envs/TF/lib/python3.7/site-packages/tornado/platform/asyncio.py", 

In [11]:
with tf.Session(graph=tf.Graph()) as sess:
    sess.run(tf.global_variables_initializer())
    tf.saved_model.loader.load(sess, ["serve"], "./testdir")
    graph = tf.get_default_graph()
    x = sess.graph.get_tensor_by_name('sent_input:0')
    y = sess.graph.get_tensor_by_name('senti_map:0')
    dp = sess.graph.get_tensor_by_name('import/LSTM/Dropout:0')
    data_X, data_Y = sentiReader.GetTrainingBatch(
                                            0, 
                                            20
                            )
    sess.run(tf.global_variables_initializer())
    tt_vals = tf.trainable_variables()
    scores = sess.run(y,
           feed_dict={x: data_X, dp:0.8})
    print(scores)


FailedPreconditionError: Attempting to use uninitialized value import/Embedding/CNN_OUT/highway_lin_1/Bias
	 [[node import/Embedding/CNN_OUT/highway_lin_1/Bias/read (defined at <ipython-input-11-dd1c849ccdd5>:3) ]]

Original stack trace for 'import/Embedding/CNN_OUT/highway_lin_1/Bias/read':
  File "/home/hadoop/.conda/envs/TF/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hadoop/.conda/envs/TF/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hadoop/.conda/envs/TF/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hadoop/.conda/envs/TF/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/hadoop/.conda/envs/TF/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/hadoop/.conda/envs/TF/lib/python3.7/site-packag

In [10]:
tt_vals

[]

In [ ]:
w2v = lstm_char_cnn.WordEmbedding(
                max_word_length = FLAGS.max_char_num , 
                char_vocab_size = char_vocab.size, 
                char_embed_size = FLAGS.char_embed_size, 
                kernels = eval(FLAGS.kernels), 
                kernel_features = eval(FLAGS.kernel_features), 
                num_highway_layers = FLAGS.highway_layers,
                embedding_dim = FLAGS.embedding_dim
            )
lstm_lm = lstm_char_cnn.LSTM_LM(
            batch_size = FLAGS.batch_size, 
            num_unroll_steps = FLAGS.num_unroll_steps, 
            rnn_size = FLAGS.rnn_size, 
            num_rnn_layers = FLAGS.rnn_layers, 
            word_vocab_size = word_vocab.size
        )

char_train_graph = lstm_char_cnn.infer_train_model(
                    w2v, lstm_lm, 
                    batch_size = FLAGS.batch_size, 
                    num_unroll_steps = FLAGS.num_unroll_steps, 
                    max_word_length = FLAGS.max_char_num, 
                    learning_rate = FLAGS.learning_rate,
                    max_grad_norm = FLAGS.max_grad_norm
                 )
#sentiment analysis model
s_model = model.SentiModel(FLAGS.hidden_dim, 5)
senti_train_graph = model.InferSentiTrainGraph(
                        w2v, 
                        s_model, 
                        max_word_num = FLAGS.max_sent_len, 
                        max_char_num = FLAGS.max_char_num, 
                        hidden_dim = FLAGS.hidden_dim, 
                        sent_num = FLAGS.sent_num,
                        embedding_dim = FLAGS.embedding_dim
                    )
# df model
rdm_model = model.RDM_Model(
                max_seq_len = FLAGS.max_seq_len, 
                max_word_num = FLAGS.max_sent_len, 
                embedding_dim = FLAGS.embedding_dim, 
                hidden_dim = FLAGS.hidden_dim
            )
rdm_train_graph = model.InferRDMTrainGraph(
                        w2v, s_model, rdm_model, 
                        max_seq_len = FLAGS.max_seq_len, 
                        max_word_num = FLAGS.max_sent_len, 
                        max_char_num = FLAGS.max_char_num, 
                        hidden_dim = FLAGS.hidden_dim, 
                        embedding_dim = FLAGS.embedding_dim,
                        class_num = FLAGS.class_num
                )

# rl model
cm_model = model.CM_Model(
                    max_word_num = FLAGS.max_sent_len, 
                    embedding_dim = FLAGS.embedding_dim, 
                    hidden_dim = FLAGS.hidden_dim, 
                    action_num = FLAGS.action_num
            )
cm_train_graph = model.InferCMTrainGraph(
                        w2v, s_model, rdm_model, cm_model, 
                        max_word_num = FLAGS.max_sent_len, 
                        embedding_dim = FLAGS.embedding_dim, 
                        hidden_dim = FLAGS.hidden_dim, 
                        action_num = FLAGS.action_num
                    )

saver = tf.train.Saver(tf.global_variables(), max_to_keep=4)
sess = tf.Session()
with sess.as_default():
    sess.run(tf.global_variables_initializer())

summary_writer = tf.summary.FileWriter(FLAGS.train_dir, graph=sess.graph)